In [ ]:
# 1. Подключаемся к Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ============================================================
# 0. CONFIG — заполняем под свой датасет
# ============================================================
DATA_FILE = "/content/drive/MyDrive/finetuning_text_pairs_3_clean_v2.csv"   # CSV, TSV или JSONL
FILE_TYPE = "csv"                  # "csv", "tsv", "jsonl", "excel"
COL_SRC   = "finetuning_text_pairs_3"              # <-- имя колонки c «оригиналом»
COL_TGT   = "Unnamed: 1"               # <-- имя колонки c «упрощением»
DROP_DUP_SRC = True                # удалять дубликаты по источнику
VAL_SPLIT    = 0.10                # доля валидации
SEED         = 42


In [ ]:
# ============================================================
# 1. Зависимости
# ============================================================
!pip -q install -U transformers peft bitsandbytes accelerate datasets evaluate \
               sentencepiece fugashi[unidic-lite] sacrebleu rouge-score pandas openpyxl pyarrow tqdm

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.1+cu118 requires torch==2.3.1, but you have torch 2.7.0 which is incompatible.
cudf-cu12 25.2.1 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 20.0.0 which is incompatible.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
# ============================================================
# 2. Читаем и приводим данные к нужному формату ----------------
# ============================================================
import pandas as pd, numpy as np, re, unicodedata, textwrap, torch, random, os
from datasets import Dataset
from sklearn.model_selection import train_test_split

In [ ]:
def read_data(path, ftype):
    if ftype == "csv":
        return pd.read_csv(path)
    if ftype == "tsv":
        return pd.read_csv(path, sep="\t")
    if ftype == "jsonl":
        return pd.read_json(path, lines=True)
    if ftype == "excel":
        return pd.read_excel(path)
    raise ValueError(f"Не понимаю FILE_TYPE={ftype}")

df = read_data(DATA_FILE, FILE_TYPE)
print("Колонки в файле:", list(df.columns))

Колонки в файле: ['finetuning_text_pairs_3', 'Unnamed: 1']


In [ ]:
# Мини-валидация, чтобы не упасть дальше
for col in (COL_SRC, COL_TGT):
    if col not in df.columns:
        raise ValueError(f"Колонка «{col}» не найдена. "
                         f"Проверьте CONFIG или названия столбцов.")

In [ ]:
# Убираем строки, в которых отсутствует одна из частей
df = df.dropna(subset=[COL_SRC, COL_TGT]).reset_index(drop=True)

# (опционально) удаляем дубли по исходнику
if DROP_DUP_SRC:
    df = df.drop_duplicates(subset=[COL_SRC]).reset_index(drop=True)

# (опционально) фильтр «слишком похожие»
def too_similar(a, b, thr=0.8):
    words_a = set(a.split())
    if len(words_a) == 0: return True
    inter = len(words_a & set(b.split())) / len(words_a)
    return (a.strip() == b.strip()) or inter > thr
df = df[~df.apply(lambda r: too_similar(r[COL_SRC], r[COL_TGT]), axis=1)].reset_index(drop=True)

print(f"После очистки осталось {len(df):,} пар.")

После очистки осталось 2,668 пар.


In [ ]:
# ============================================================
# 2. Загрузка модели и токенизатора
# ============================================================
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

MODEL_ID = "Vikhrmodels/Vikhr-Llama-3.2-1B-Instruct"

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tok = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
tok.pad_token = tok.eos_token  # для паддинга

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    quantization_config=bnb_cfg,
    torch_dtype=torch.bfloat16,
)

# LoRA-настройки
lora_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


trainable params: 3,407,872 || all params: 1,239,222,272 || trainable%: 0.2750


In [ ]:
assert {COL_SRC, COL_TGT}.issubset(df.columns)

In [ ]:
# ============================================================
# 4. Токенизация  (Патч: надёжные имена + короче seq_len)
# ============================================================

### 4.0  Гарантируем, что в датафрейме остались "src" и "tgt"
df = df.rename(columns={COL_SRC: "src", COL_TGT: "tgt"})
COL_SRC, COL_TGT = "src", "tgt"          # дальше используем только их

### 4.1  Снова строим Dataset из уже переименованного df
train_df, val_df = train_test_split(df, test_size=VAL_SPLIT, random_state=SEED)
train_ds, val_ds = map(Dataset.from_pandas, (train_df, val_df))

PROMPT = "Упрости текст, сохраняя смысл:\n\n{src}\n\nУпрощённая версия:"
MAXLEN = 1024          # как решили выше

def tokenize(batch):
    input_ids, labels, attention = [], [], []

    for src, tgt in zip(batch[COL_SRC], batch[COL_TGT]):
        # --- 1) токены
        prompt_ids = tok(
            PROMPT.format(src=src), add_special_tokens=False
        ).input_ids
        target_ids = tok(
            tgt, add_special_tokens=False
        ).input_ids + [tok.eos_token_id]

        ids = prompt_ids + target_ids
        # --- 2) обрезаем до MAXLEN
        ids = ids[:MAXLEN]

        # --- 3) labels: -100 на промпт, реальные токены на таргет
        label_ids = [-100] * len(prompt_ids) + target_ids
        label_ids = label_ids[:MAXLEN]

        input_ids.append(ids)
        labels.append(label_ids)
        attention.append([1] * len(ids))         # attention_mask

    return {
        "input_ids":     input_ids,
        "labels":        labels,
        "attention_mask": attention,
    }

# заново строим датасеты
train_ds = train_ds.map(tokenize, batched=True, remove_columns=train_ds.column_names)
val_ds   = val_ds.map(tokenize, batched=True, remove_columns=val_ds.column_names)

Map:   0%|          | 0/2401 [00:00<?, ? examples/s]

Map:   0%|          | 0/267 [00:00<?, ? examples/s]

In [ ]:
# ============================================================
# 5. Обучение --------------------------------------------------
# ============================================================
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

args = TrainingArguments(
    output_dir="/content/llama_lora",
    num_train_epochs=8,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=16,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    bf16=False,
    fp16=True,
    report_to="none",
)

collator = DataCollatorForSeq2Seq(tok, model=model, label_pad_token_id=-100, pad_to_multiple_of=8)
trainer = Trainer(model=model, args=args,
                  train_dataset=train_ds, eval_dataset=val_ds,
                  data_collator=collator)
trainer.train()
model.save_pretrained("/content/drive/MyDrive/llama_lora/adapter")

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,1.013100,0.862254
2,0.777900,0.815085
3,0.718000,0.797019
4,0.660300,0.787400
5,0.614600,0.788001
6,0.589700,0.791764
7,0.562600,0.796540


In [ ]:
# ============================================================
# 6. Быстрая проверка (10 примеров + ROUGE-L & chrF)
# ============================================================
import evaluate, textwrap, random
rouge = evaluate.load("rouge")
chrf  = evaluate.load("chrf")

def gen(text, temp=0.3, top_p=0.9, max_new=256):
    prompt = PROMPT.format(src=text)
    ids = tok(prompt, return_tensors="pt").to(model.device)
    out = model.generate(**ids, do_sample=True, temperature=temp,
                         top_p=top_p, max_new_tokens=max_new,
                         eos_token_id=tok.eos_token_id)
    return tok.decode(out[0], skip_special_tokens=True)\
             .split("Упрощённая версия:")[-1].strip()

sample = val_df.sample(10, random_state=SEED)
preds  = [gen(t) for t in sample[COL_SRC]]

print("\n=== DEMO ===")
for i, (src, ref, pr) in enumerate(zip(sample[COL_SRC], sample[COL_TGT], preds), 1):
    print(f"\n--- #{i}\nSRC: {textwrap.shorten(src, 2000)}"
          f"\nREF: {textwrap.shorten(ref, 500)}"
          f"\nPRD: {textwrap.shorten(pr,  500)}")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.



=== DEMO ===

--- #1
SRC: RUS Информация для предварительного ознакомления. Официальной информацией изготовителя не является 1.6 В машине предусмотрена система электронного контроля, которая способствует равномерности распределения белья в барабане перед каждым отжимом для исключения повышенных вибраций и шума. 1.7 В машине (в зависимости от модели) предусмотрена защитная система в наливном шланге, которая определяет утечку воды при повреждении шланга подача воды из водопровода в машину прекращается, стирка останавливается. 1.8 Дополнительная защитная система Аквастоп от утечки воды внутри машины предусмотрена в модели машины с буквой -Ав обозначении. Защитная система прекращает подачу воды из водопровода при повреждении машины стирка останавливается, вода из бака сливается и на дисплее высвечивается показание неисправности. 1.9 Эксплуатировать машину необходимо: при температуре окружающей среды (25 10) 0С и относительной влажности не более 75 %; при давлении в водопроводной сети от 0

In [ ]:
metrics = rouge.compute(predictions=preds, references=sample[COL_TGT], use_stemmer=True)
print("\nROUGE-L:", round(metrics["rougeL"], 4),
      " | chrF:", round(chrf.compute(predictions=preds, references=sample[COL_TGT]), 4))

KeyError: 0

In [ ]:
# ============================================================
# 7. Функция simplify() для ручных тестов
# ============================================================
def simplify(text, temp=0.3):
    return gen(text, temp=temp)

print("\n=== Ручная проверка ===")
print(simplify("Инструкция по организации делопроизводства при осуществлении внутреннего документооборота Настоящим уведомляем всех причастных лиц о нижеследующем порядке обращения с документацией и ведения учетных записей: 1. Вся входящая корреспонденция подлежит обязательной регистрации в течение одного рабочего дня с момента поступления таковой в уполномоченное подразделение. 2. В целях оптимизации процесса согласования служебных записок надлежит осуществлять визирование документов в строгом соответствии с утвержденным регламентом, при этом в обязательном порядке проставляя отметку о дате ознакомления. 3. Ответственность за своевременную подготовку отчетной документации возлагается на руководителей структурных подразделений, осуществляющих контроль за исполнением настоящей инструкции. 4. При выявлении случаев ненадлежащего оформления документов, работнику, допустившему нарушение, необходимо в срок, не превышающий трех рабочих дней, обеспечить устранение выявленных недостатков и представить доработанный вариант на повторное рассмотрение. Несоблюдение вышеуказанных требований может повлечь применение мер дисциплинарного воздействия в отношении лиц, виновных в нарушении "))

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.



=== Ручная проверка ===
Инструкция по организации делопроизводства при внутреннем документооборе

Настоящим вы уведомляете всех участников процесса о том, как будет организован регистрация входящих документов и управление учетными записями.

1. Входящая документация должна быть регистрирована в течение одного рабочего дня после ее поступления в соответствующее отделение.
2. Для оптимизации процесса согласования служебных записок следует проводить визуализацию документов в соответствии с установленным регламентом, при этом необходимо отметать дату получения документа.
3. Владельцы отчетной документации должны быть заинтересованы в своевременном подготовке документов и должны следовать указанным требованиям.
4. При обнаружении несоответствия документам, допущенных нарушений, работник, который его совершал, должен в течение трех рабочих дней восстановить документы и представить их для повторного рассмотрения. Нарушение требований может привести к дисциплинарному санкционированию лиц, нес

# Генерация для сравнения с золотым стандартом

In [ ]:
BASE_MODEL_ID = "Vikhrmodels/Vikhr-Llama-3.2-1B-Instruct"
ADAPTER_DIR   = "/content/drive/MyDrive/llama_lora/adapter"          # ⚑ ваш сохранённый LoRA-чек-пойнт
SRC_XLSX      = "/content/drive/MyDrive/unique_texts_200_formatted_golden_standard.xlsx"
DST_XLSX      = "/content/drive/MyDrive/unique_texts_200_formatted_golden_standard_vikhr_llama.xlsx"

In [ ]:
# 1️⃣  убираем всё, что конфликтует
!pip uninstall -y torch torchvision torchaudio bitsandbytes

# 2️⃣  ставим согласованный стек CUDA-11.8  (PyTorch 2.2.x  +   bnb 0.43.1)
!pip install --no-cache-dir torch==2.2.2+cu118 torchvision==0.17.2+cu118 torchaudio==2.2.2+cu118 \
            --index-url https://download.pytorch.org/whl/cu118
!pip install --no-cache-dir bitsandbytes==0.43.1

# 3️⃣  остальные зависимости
!pip install -U transformers accelerate peft datasets

Found existing installation: torch 2.7.0
Uninstalling torch-2.7.0:
  Successfully uninstalled torch-2.7.0
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: bitsandbytes 0.45.5
Uninstalling bitsandbytes-0.45.5:
  Successfully uninstalled bitsandbytes-0.45.5
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.2/819.2 MB 232.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 271.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 116.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 243.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 199.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
import torch, bitsandbytes as bnb, subprocess, os
print("torch  :", torch.__version__)
print("cuda   :", torch.version.cuda)
print("bnb    :", bnb.__version__)
print("GPU OK :", torch.cuda.is_available(), torch.cuda.get_device_name(0))


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

torch  : 2.2.2+cu118
cuda   : 11.8
bnb    : 0.43.1
GPU OK : True Tesla T4


In [ ]:
# 1️⃣ убрать старые сборки
!pip uninstall -y torch torchvision torchaudio bitsandbytes

# 2️⃣ поставить PyTorch 2.3 + CUDA-11.8
!pip install --no-cache-dir "torch==2.3.1+cu118" "torchvision==0.18.1+cu118" \
                             "torchaudio==2.3.1+cu118" \
             --index-url https://download.pytorch.org/whl/cu118

# 3️⃣ bitsandbytes под ту же CUDA
!pip install --no-cache-dir bitsandbytes==0.43.1

# 4️⃣ остальное
!pip install -U transformers accelerate peft datasets

Found existing installation: torch 2.2.2+cu118
Uninstalling torch-2.2.2+cu118:
  Successfully uninstalled torch-2.2.2+cu118
Found existing installation: torchvision 0.17.2+cu118
Uninstalling torchvision-0.17.2+cu118:
  Successfully uninstalled torchvision-0.17.2+cu118
Found existing installation: torchaudio 2.2.2+cu118
Uninstalling torchaudio-2.2.2+cu118:
  Successfully uninstalled torchaudio-2.2.2+cu118
Found existing installation: bitsandbytes 0.43.1
Uninstalling bitsandbytes-0.43.1:
  Successfully uninstalled bitsandbytes-0.43.1
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 839.7/839.7 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.9/142.9 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 64.9 MB/s eta 0:00:00
  

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 259.6 MB/s eta 0:00:00


In [ ]:
# обновляем только bnb (остальное трогать не нужно)
!pip install --no-cache-dir --force-reinstall "bitsandbytes>=0.43.2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 212.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 235.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 214.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 295.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 192.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 242.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 312.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 273.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 268.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 399.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 177.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch, bitsandbytes as bnb
print(torch.__version__, torch.version.cuda)   # 2.3.x  11.8
print("bnb", bnb.__version__)                  # ≥ 0.43.2

2.7.0+cu126 12.6
bnb 0.45.5


In [ ]:
# ============================================================
# 1. Загрузка модели с LoRA-весами
# ============================================================
import torch, pandas as pd, numpy as np, textwrap, os, gc
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16",
)

tok = AutoTokenizer.from_pretrained(BASE_MODEL_ID, use_fast=True)
tok.pad_token = tok.eos_token

base = AutoModelForCausalLM.from_pretrained(
    BASE_ID,
    quantization_config=bnb_cfg,
    device_map="auto",    # отправит блоки сразу на cuda:0
    torch_dtype="float16",
)

model = PeftModel.from_pretrained(base, ADAPTER_DIR)
model.eval()                       # на всякий случай
device = next(model.parameters()).device

PROMPT = "Упроси текст, сохраняя смысл:\n\n{src}\n\nУпрощённая версия:"
@torch.inference_mode()
def simplify(text, temp=0.3, max_new=256):
    prompt = PROMPT.format(src=text)
    ids = tok(prompt, return_tensors="pt").to(device)
    gen = model.generate(
        **ids,
        do_sample=True, temperature=temp, top_p=0.9,
        max_new_tokens=max_new,
        eos_token_id=tok.eos_token_id,
    )
    return tok.decode(gen[0], skip_special_tokens=True)\
             .split("Упрощённая версия:")[-1].strip()

RuntimeError: Failed to import transformers.models.bloom.modeling_bloom because of the following error (look up to see its traceback):
operator torchvision::nms does not exist

In [ ]:



# ============================================================
# 2. Читаем исходный Excel или продолжаем ранее созданный
# ============================================================
df = pd.read_excel(SRC_XLSX)
if "finetuned_model" not in df.columns:
    df["finetuned_model"] = np.nan      # создаём пустой столбец

# ============================================================
# 3. Генерация с поэтапным сохранением
# ============================================================
for idx, row in df.iterrows():
    if pd.isna(row["finetuned_model"]):          # пропускаем уже сгенерированное
        try:
            df.at[idx, "finetuned_model"] = simplify(str(row.iloc[0]))  # первый столбец
            print(f"{idx+1}/{len(df)} ✓")
        except Exception as e:
            print(f"{idx+1} ✗ {e}")              # логируем и идём дальше
        # сохраняем после КАЖДОЙ генерации
        df.to_excel(DST_XLSX, index=False)

print("Готово! Файл сохранён:", DST_XLSX)